In [30]:
from openai import AzureOpenAI, OpenAI
import openai
from dotenv import load_dotenv
load_dotenv('azure.env')
import os
from openai import OpenAI
from promptflow.tracing import start_trace

# instrument OpenAI
start_trace()

client = AzureOpenAI(api_version="2024-03-01-preview", 
                     azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'], 
                     azure_deployment='gpt-4-1106-preview')

Starting prompt flow service...


[2024-05-09 15:53:35,764][promptflow][WARNING] - The starting prompt flow process did not finish within the timeout period. Kindly reminder: If you have previously upgraded the prompt flow package , please double-confirm that you have run 'pf service stop' to stop the prompt flowservice before proceeding with the upgrade. Otherwise, you may encounter unexpected environmental issues or inconsistencies between the version of running prompt flow service and the local prompt flow version. Alternatively, you can use the 'pf upgrade' command to proceed with the upgrade process for the prompt flow package.


You can stop the prompt flow service with the following command:'pf service stop'.
Alternatively, if no requests are made within 1 hours, it will automatically stop.


In [23]:
from langchain_experimental.tools import PythonAstREPLTool
from colorama import Fore
import re

class PythonREPL:
    def __init__(self):
        self.local_vars = {}
        self.python_tool = PythonAstREPLTool()
    def run(self, code: str) -> str:
        code = extract_code_from_block(code)
        output = str(self.python_tool.run(code))
        if output == "":
            return "Your code is executed successfully"
        else:
            return output

python_repl = PythonREPL()

def get_tool_info(tool_name):
    tools = {
        "python": {"type": "tool", 
                   "name": "python", 
                   "use": "Use this to execute python code. Display your results using the print function.", 
                   "input": "Input should be a valid python code. Ensure proper indentation", 
                   "function": python_repl.run},
        }
    return tools[tool_name]



tools = []
value_dict = {}
tools_description = "\n\nYou can use the following actions:\n\n" 
choice = 'python'
tools.append(choice)
tool_info = get_tool_info(choice)
tools_description = tools_description + "Action Name: " + tool_info["name"] + "\nWhen To Use: " + tool_info["use"] + "\nInput: " + tool_info["input"]
tools_description = tools_description + "\n\n"
value_dict[choice] = tool_info["function"]
value_dict

{'python': <bound method PythonREPL.run of <__main__.PythonREPL object at 0x000002751CD71D90>>}

In [24]:
def run(content, instruction):
    content = content.replace('<<instruction>>', instruction)
    count = 0
    while(True):
        count = count + 1
        if count > 5:
            raise ValueError("Too many steps")
        #print(Fore.BLUE + content)
        output = llm(content)
        output = output.replace("\nObservation:", "")
        print(Fore.MAGENTA + output)
        regex = r"Action\s*\d*\s*:(.*?)\nInput\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, output, re.DOTALL)

        if "Final Answer:" in output and not match:
            break
            print(output.split("Final Answer:")[1])

        if "Step" not in output:
            print(Fore.YELLOW + "The model didnt output a step.")
            output = "Please follow the format Step/Reason/Action/Input/Observation"
            content = content + "\n" + output
            continue

        if "Reason" not in output:
            print(Fore.YELLOW + "The model didnt output a reason.")
            output = "Please follow the format Step/Reason/Action/Input/Observation"
            messages = content + "\n" + output
            continue

        if output.count("Input") > 1:
            print(Fore.YELLOW + "The model went crazy.")
            output = "Please go one step at a time."
            content = content + "\n" + output
            continue

        if not match:
            print(Fore.RED + "The model was sidetracked.")
            output = "You are not following the format. Please follow the given format."
            messages = [{"role": "user", "content": content + "\n" + output}]
            continue


        action = match.group(1).strip().lower()
        if action not in tools:
            output = f"Invalid Action. Your action should be one of {tools}."
            print(Fore.YELLOW + "The agent forgot his tools." + output) 
            content = content + "\n" + output
            continue

        action_input = match.group(2)
        match = re.search(r"Step (\d+): (.*)", output)
        step_number = int(match.group(1)) + 1
        observation = value_dict[action](action_input)
        print(Fore.GREEN + "\nObservation: " + str(observation))
        output = output + "\nObservation: " + str(observation)
        content = content + "\n" + output

In [25]:
content = """You can use the following actions:

Action Name: python
When To Use: When you send a message containing Python code to python, it will be executed in a stateful Jupyter notebook environment. python will respond with the output of the execution
seconds.
Input: Input should be a valid python code. Ensure proper indentation

Acomplish the task in steps. If you get error in previous step fix it in the current step. Use the following format:

Step 1: The first step
Reason: Reason for taking this step
Action: the action to take, should be one of ['python'].
Input: the input to the action
Observation: the result of the action

Step 2: The second step
Reason: Reason for taking this step
Action: the action to take, should be one of ['python'].
Input: the input to the action 
Observation: the result of the action

... (this Step/Reason/Action/Input/Observation repeats for all steps)

Once you have completed all the steps, your final answer should be in the format:
Final Answer: I have completed all the steps

Begin

<<instruction>>
"""

In [26]:
def llm(user_input):
    response = client.chat.completions.create(
        model='gpt-4-1106-preview',
        messages=[
            {'role': 'user', 'content': user_input}
        ],
        temperature=0,
        stream=False,
        #stream_options={"include_usage": True}, # retrieving token usage for stream response
    )
    return response.choices[0].message.content# a ChatCompletion request

In [27]:
def extract_code_from_block(response):
    if '```' not in response:
        return response
    if '```python' in response:
        code_regex = r'```python(.+?)```'
    else:
        code_regex = r'```(.+?)```'
    code_matches = re.findall(code_regex, response, re.DOTALL)
    code_matches = [item for item in code_matches]
    return  "\n".join(code_matches)

def fix_error(code, result):
    count = 0
    while "Your code has the following error." in result:
        error = result.replace("Your code has the following error. Please provide the corrected code.", "")
        user_input = f"""Here is the Code and the Error. 

Code:
{code}

Error:
{error}

Fix the given using the following format:

Explanation: Explain the error in the code

Corrected Code: Put the Corrected Code here"""
        print(Fore.RED + "Code needs some correction.\n" )
        code = llm(user_input)
        code = code[code.rfind('Corrected Code:') + len("Corrected Code:"):]
        code = extract_code_from_block(code)
        print(Fore.CYAN + "Corrected Code.\n" + code)
        result = python_repl.run(code)
        print(Fore.BLUE + "Result.\n" + result)
        count += 1
        if count > 5:
            raise ValueError("Too many steps")
    print(Fore.GREEN + "Code has been corrected.\n" + code)
    return result

In [ ]:
def execute(instruction):
    return run(content,instruction)
command = """print the glimpse of iris.csv.
Preprocess the data before trainig the model.
Train a random forest model to predict the Species column.
Save the classification report in result.txt"""
execute(command)